# Optimization Journey
Claus Brenner, 17.12.2020.

# Fitting a model - first, in 1D

## Least squares estimation in 1D
Say we are given these points $b_1, b_2, b_3, b_4$:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'png'
# Uncomment the following two lines to get large plots.
#plt.rcParams['figure.figsize'] = (11, 6)
#plt.rcParams['figure.dpi'] = 150

# Very few selected points.
b_1d = np.array([3, 4, 5, 7])
plt.plot(b_1d, np.zeros_like(b_1d), "bo");
plt.xlim(0,10);

...and search for the "least squares solution", i.e., the $x$ that has the smallest least squares distance from all $b_i$.:

$$x^*=\arg\min_{x\in\mathbb{R}} \sum_i (x-b_i)^2$$

That means, we have to vary $x\in\mathbb{R}$ until the sum becomes minimal.

The reason why we would want to do this may be e.g. that we want to find a maximum likelihood estimation of $x$, given linear measurements corrupted by Gaussian measurement errors.

So we sum up a set of quadratic functions $(x-b_i)^2$:

In [ ]:
x_vals_1d = np.linspace(0, 10, num=1000)
for b in b_1d: plt.plot(x_vals_1d, (x_vals_1d-b)**2)
plt.plot(b_1d, np.zeros_like(b_1d), "bo")
plt.ylim(-.5,20);

Maybe I could trick somebody to think the sum must have a complex shape, like this:

In [ ]:
plt.plot(x_vals_1d, ((x_vals_1d-b_1d[:,np.newaxis])**2).min(axis=0), "black")
plt.plot(b_1d, np.zeros_like(b_1d), "bo")
plt.ylim(-.5,20);

...but no, the sum is of course a *quadratic function*. It can't be otherwise, because adding up quadratic terms can't result in, say, cubic terms.

In [ ]:
for b in b_1d: plt.plot(x_vals_1d, (x_vals_1d-b)**2)
plt.plot(x_vals_1d, ((x_vals_1d-b_1d[:,np.newaxis])**2).sum(axis=0), "black")
plt.plot(b_1d, np.zeros_like(b_1d), "bo")
plt.ylim(-.5,20);

Since the sum is a quadratic function (with positive $x^2$ coefficients), it is of course also a **convex** function. Since the *domain* of $f(x)$ is *convex* (the full $\mathbb{R}$), and the *function* $f(x)$ itself is *convex*, the minimum can be found efficiently, for example, by "walking downhill".

In general, what we have here is a case of **convex optimization**, see e.g. Boyd and Vandenberghe (bibliography at end).

In this case, it is even easier, and we need not iteratively "walk downhill". The quadratic function has a global minimum where the gradient is zero. In 1D, the gradient is the derivative w.r.t. $x$, so $$f'(x)=\frac{d}{dx}\left(\sum_i (x-b_i)^2\right) = \sum_i 2(x-b_i).$$ Then, setting $\sum_i (x-b_i) = 0$ yields $mx - \sum_i b_i = 0$, so the result is the mean:

$$x^* = \frac{1}{m}\sum_i b_i.$$

In our example, the mean is:

In [ ]:
np.mean(b_1d)

In [ ]:
for b in b_1d: plt.plot(x_vals_1d, (x_vals_1d-b)**2)
plt.plot(x_vals_1d, ((x_vals_1d-b_1d[:,np.newaxis])**2).sum(axis=0), "black")
plt.plot(b_1d, np.zeros_like(b_1d), "bo")
plt.axvline(x=np.mean(b_1d), c="k", ls="--")
plt.ylim(-.5,20);

The result is indicated by the dotted line. All this says is that the least squares estimate of $\mu$ is the mean, and the mean of this four numbers is this certain value.

However, if the task is to estimate an unknown value from measurements $b_i$, then it could be that the unknown value is $4$, and the measurement process added some noise, which has lead to the measurements $3$, $4$ and $5$, whereas the value $7$ is actually an outlier.


### Checking with random samples and outliers.
Let's say our measurement is normal distributed with $\mu=4$ and $\sigma=0.1$.
(Note the points are all on a line. Some jitter in y is added for illustration only.)

In [ ]:
rng = np.random.default_rng(12345)
n_1d_meas = 30
b_1d_meas = rng.normal(4, .1, n_1d_meas)

def report_mean(inliers, outliers):
    both = np.concatenate((inliers, outliers))
    plt.plot(inliers, rng.uniform(-.2,.2,len(inliers)), "b.")
    plt.plot(outliers, rng.uniform(-.2,.2,len(outliers)), "r.")
    plt.axvline(x=np.mean(both), c="silver", ls="--")
    plt.ylim(-1,1)
    plt.xlim(-2,10)
    print("inliers %d outliers %d mean %.3f" % (len(inliers), len(outliers), np.mean(both)))

report_mean(b_1d_meas, [])

Then, our mean works nicely as an estimator for $\mu$. Now if we have "outliers", which are uniformly distributed:

In [ ]:
n_1d_outl = 25
b_1d_outl = rng.uniform(-2,10,n_1d_outl)
report_mean(b_1d_meas, b_1d_outl)

We see that our estimator may be influenced by the outliers. In this case, the influence is not very large, since "data" and "outliers" both are centered at the same $\mu$.

A more realistic case could assume that all outliers are on one side. E.g., in Laserscanning, the measured ranges may be mostly too short, but (almost) never too long. So now, we assume the outliers are only to the right, uniformly distributed. In this "asymmetric" case we get a skewed overall distribution.

In [ ]:
b_1d_outl_a = np.abs(b_1d_outl-4)+4
report_mean(b_1d_meas, b_1d_outl_a)

## Least absolute deviations: the $l_1$ norm
In least squares, outliers have a large influence on the result, because the value added to the sum increases quadratically.

On can play with the function which "weights" how small and large deviations influence the estimation result. One can generalize

$$x^* = \arg\min_x \sum_i (x-b_i)^2$$
to
$$x^* = \arg\min_x \sum_i \rho(x-b_i)$$

where $\rho(r)=r^2$ for the least squares case (define $r$ to be the residual, $r_i=(x-b_i)$). $\rho$ is called a **loss function** and these estimators are called **M-estimators**, see [@wikipedia](https://en.wikipedia.org/wiki/M-estimator).

For example, in order to make the influence of large values smaller, and small values larger, one can use the function $\rho(r)=|r|$, leading to:

$$x^* = \arg\min_x \sum_i |x-b_i|,$$
or in general,
$$x^* = \arg\min_x ||Ax-b||_1.$$

For our initial 1D example, we now get:

In [ ]:
for b in b_1d: plt.plot(x_vals_1d, np.abs(x_vals_1d-b))
plt.plot(b_1d, np.zeros_like(b_1d), "bo")
plt.ylim(-.5,10);

Now we can again guess the shape of the resulting sum. Will it be convex?

In [ ]:
for b in b_1d: plt.plot(x_vals_1d, np.abs(x_vals_1d-b))
plt.plot(x_vals_1d, (np.abs(x_vals_1d-b_1d[:,np.newaxis])).sum(axis=0), "black")
plt.plot(b_1d, np.zeros_like(b_1d), "bo")
plt.ylim(-.5,10);

It *is* convex. We could have known that beforehand, because $\rho(x)=|x|$ is convex, and *any nonnegative weighted sum of convex functions is convex* (see Boyd & Vandenberghe, sect. 3.2.1). So there is only one global minimum.

However, we see that there is a plateau between $b_1$ and $b_2$. So while there is a global minimum value, its location $x^*$ is not unique.

How can we find the minimum? We can walk downhill. Say, we start at 0, so we are in "the negative section" of all four curves, and the slope of the black curve must be $-4$. So walking downhill means, walking to the right. If we walk to the right, the next change will occur at $b_0$, where the slope of the corresponding $|x-b_0|$ curve changes from $-1$ to $+1$, so the slope of the black curve changes from $-4$ to $-2$. We continue to the right, and at $b_1$ the slope changes from $-2$ to $0$. We are at the global minimum.

If we think about it, we do not have to do all of this walking. For any given point $x$, all $b_i$ to the left of it contribute $+1$ to the slope, and all $b_i$ to the right of it contribute $-1$. So we have zero if the number of $b_i$ to the left and $b_i$ to the right are equal. Therefore, the least absolute deviations estimator is the median:

$$x^* = \mbox{median}_i(d).$$

If $m$, the number of observations $b_i$, is odd, the median will be in the central $b_{(m-1)/2}$, whereas if it is equal it is not uniquely defined, but usually, the mean between $b_{(m-2)/2}$ and $b_{(m-2)/2+1}$ is returned. For example, in numpy, we get the following (dotted black line):

In [ ]:
for b in b_1d: plt.plot(x_vals_1d, np.abs(x_vals_1d-b))
plt.plot(x_vals_1d, (np.abs(x_vals_1d-b_1d[:,np.newaxis])).sum(axis=0), "black")
plt.plot(b_1d, np.zeros_like(b_1d), "bo")
plt.axvline(x=np.median(b_1d), c="k", ls="--")
plt.ylim(-.5,10);

### Checking with our 1D outlier example.
First, without outliers, the median works great.

In [ ]:
def report_median(inliers, outliers):
    both = np.concatenate((inliers, outliers))
    plt.plot(inliers, rng.uniform(-.2,.2,len(inliers)), "b.")
    plt.plot(outliers, rng.uniform(-.2,.2,len(outliers)), "r.")
    plt.axvline(x=np.mean(both), c="silver", ls="--", label="mean")
    plt.axvline(x=np.median(both), c="dimgray", ls="--", label="median")
    plt.legend()
    plt.ylim(-1,1)
    plt.xlim(-2,10)
    print("inliers %d outliers %d mean %.3f median %.3f" %\
          (len(inliers), len(outliers), np.mean(both), np.median(both)))
report_median(b_1d_meas, [])

With our "symmetric outliers", we usually get a better result (dark gray line) than with the mean (light gray line). (But with the small number of samples used here, it depends a bit on the actual samples.)

In [ ]:
report_median(b_1d_meas, b_1d_outl)

In the "asymmetric outlier" case, the median is much better, although it starts to walk away from the true value.

In [ ]:
report_median(b_1d_meas, b_1d_outl_a)

However, if we have even more outliers, we get:

In [ ]:
n_1d_more_outl = 2*n_1d_meas
b_1d_more_outl_a = rng.uniform(4, 10, n_1d_more_outl)
report_median(b_1d_meas, b_1d_more_outl_a)

Intuitively, for each two more outliers to the right, the median will "walk one $b_i$ to the right", no matter how far away the added outlier to the right actually is.

## Maximum Consensus: maximizing inliers, or minimizing outliers

Now we have seen that the $l_1$ norm is more robust than the $l_2$ norm. Intuitively, with the $l_2$ norm, the actual value of the outlier plays a role. The larger it is, the more it pulls $x^*$ towards it.

In contrast, with the $l_1$ norm, each outlier only "counts", independently of its actual value. If there are many inliers and only a few outliers, they will be unable to worsen the estimate substantially, no matter how far away they are from the true value.

But if the number of outliers gets very large, the $l_1$ norm will walk away from the true solution, especially if the outliers are not "symmetric" around the true value. (Only the difference count between "left" and "right" outliers plays a role -- two more "right" outliers than "left" outliers make the median walk one to the right.) In order to further reduce the influence of outliers, we could try to discard them altogether.

The idea of **maximum consensus** is to define an "acceptance range" $[-\varepsilon, \varepsilon]$ inside which an observation counts as inlier, otherwise it is an outlier. The goal is then to find the parameters $x^*$ which maximize the number of inliers. To formulate this, the indicator function $\mathbb{1}(\cdot)$ is typically used, where $\mathbb{1}(c)$ is 1 if condition $c$ is true, and 0 otherwise. Then, the maximum consensus goal can be formulated as

$$x^* = \arg\max_x \sum_i \mathbb{1}\left( |x-b_i| \leq \varepsilon \right),$$

i.e., we are counting the inliers, $+1$ if inside, and $0$ if outside.

However, to make it fit a bit more to our previous considerations, we can define it the opposite way: we count the outliers, i.e. $0$ if inside, and $+1$ if outside. Thus, we can achieve our goal by defining the *loss function*

$$\rho(r) = \left\{ \begin{matrix}0&\mbox{for}&|r|\leq\varepsilon\\1&\mbox{else.}\end{matrix}\right.$$

Here is a comparison of loss functions $\rho(r)$ we used so far:

In [ ]:
# This is our new rho function.
def rho_1d(r, eps):
    return (np.abs(r)>eps).astype(float)

x_comparison = np.linspace(-2,2,1000)
plt.plot(x_comparison, x_comparison**2)
plt.plot(x_comparison, np.abs(x_comparison))
plt.plot(x_comparison, rho_1d(x_comparison, 1))
plt.ylim(-.1,3);

While we flattened the curve outside, we notice it is **not convex** anymore. This brings with it substantial trouble, as we will see. Because we hypothesized above that $4$ may be the correct value and $3$ and $5$ are still to be considered as inliers as well we will use $\varepsilon=1.2$.

In [ ]:
eps_1d = 1.2
for b in b_1d: plt.plot(x_vals_1d, rho_1d(x_vals_1d-b, eps_1d))
plt.plot(x_vals_1d, (rho_1d(x_vals_1d-b_1d[:,np.newaxis], eps_1d)).sum(axis=0), "black")
plt.plot(b_1d, np.zeros_like(b_1d), "bo")
plt.ylim(-.5,6);

The result has its minimum at the "right" location. It is a plateau, as in the $l_1$ case. However, as the single functions are not convex, their sum is neither. Apart from having steps, it has local minima. And of course, some of them may be identical to the global minimum, as shown in this example with two more points.

In [ ]:
b_1d_more = np.array([3,4,5,7,7.5,8])
for b in b_1d_more: plt.plot(x_vals_1d, rho_1d(x_vals_1d-b, eps_1d))
plt.plot(x_vals_1d, (rho_1d(x_vals_1d-b_1d_more[:,np.newaxis], eps_1d)).sum(axis=0), "black")
plt.plot(b_1d_more, np.zeros_like(b_1d_more), "bo")
plt.ylim(-.5,7);

### Programming a function to find the consensus.
In the least squares case, we could use $x^*=$ `mean()`, in the $l_1$ case, $x^*=$ `median()`, but the maximum consensus case is less clear.

The most obvious idea is to use a "sweep type" algorithm:
- sweep the real line from left to right
- at each $b_i-\varepsilon$ ("interval start"), decrease the total score by 1
- at each $b_i+\varepsilon$ ("interval end"), increase the total score by 1
- computing the cumulative sum, one can find the global minimum
- for now, if there are several minima, just return one of them
- to make it look nicer, return the "center of the plateau" (similar to the median case with an even number of values).

In [ ]:
def cons_1d(b_vals, eps):
    # Note tuple sort will always place the -1 event in front of the +1 event
    # so the "less or equal" is implemented correctly if events coincide.
    events = sorted([(b-eps,-1) for b in b_vals]+[(b+eps,1) for b in b_vals])
    score = np.cumsum(np.array([e[1] for e in events]))
    xs = np.argmin(score)  # Get one of the minima.
    return (events[xs][0]+events[xs+1][0])/2  # Return the center of the plateau.

Check for our simple 1D example:

In [ ]:
for b in b_1d: plt.plot(x_vals_1d, rho_1d(x_vals_1d-b, eps_1d))
plt.plot(x_vals_1d, (rho_1d(x_vals_1d-b_1d[:,np.newaxis], eps_1d)).sum(axis=0), "black")
plt.plot(b_1d, np.zeros_like(b_1d), "bo")
plt.axvline(x=cons_1d(b_1d, eps_1d), c="k", ls="--")
plt.ylim(-.5,6);

### Checking with the 1D random samples.
Since we assumed our original signal to be normal distributed with $\sigma=0.1$, we could set $\varepsilon=2\cdot\sigma=0.2$.

In [ ]:
eps_1d_random = 0.2
def report_cons(inliers, outliers, eps):
    both = np.concatenate((inliers, outliers))
    plt.plot(inliers, rng.uniform(-.2,.2,len(inliers)), "b.")
    plt.plot(outliers, rng.uniform(-.2,.2,len(outliers)), "r.")
    plt.axvline(x=np.mean(both), c="silver", ls="--", label="mean")
    plt.axvline(x=np.median(both), c="dimgray", ls="--", label="median")
    plt.axvline(x=cons_1d(both, eps), c="black", ls="--", label="max cons")
    plt.legend()
    plt.ylim(-1,1)
    plt.xlim(-2,10)
    print("inliers %d outliers %d mean %.3f median %.3f consensus %.3f" %\
          (len(inliers), len(outliers), np.mean(both), np.median(both),
           cons_1d(both, eps_1d_random)))
report_cons(b_1d_meas, [], eps_1d_random)

So, looks good with no noise. Now, for "symmetric" noise:

In [ ]:
report_cons(b_1d_meas, b_1d_outl, eps_1d_random)

...looks good, too. For the asymmetric case:

In [ ]:
report_cons(b_1d_meas, b_1d_outl_a, eps_1d_random)

...looks still good. Now, for the case with the large amount of outliers:

In [ ]:
report_cons(b_1d_meas, b_1d_more_outl_a, eps_1d_random)

...the maximum consensus occurs close to $4$. That is pretty impressive, given that we have twice as many outliers than inliers.

In order to figure out "how sure" we are about the maximum consensus, or the minimum in our score, we can plot the score over the entire range.

In [ ]:
x_vals_mc = np.linspace(3,11,1000)
all_b = np.concatenate((b_1d_meas, b_1d_more_outl_a))
vals = np.sum([rho_1d(x_vals_mc - b, eps_1d_random) for b in all_b], axis=0)
plt.plot(b_1d_meas, rng.uniform(-1,1,len(b_1d_meas))+55, "b.")  # +55 just for nice plot.
plt.plot(b_1d_more_outl_a, rng.uniform(-1,1,len(b_1d_more_outl_a))+55, "r.")
plt.plot(x_vals_mc, vals, c="k");

# Fitting a model - the multidimensional case
## Least squares estimation for the line

To illustrate, we will estimate a line. Our true line is $y = 0.5\cdot x + 2$, but in order to escape notational clashes, we will use $(p,q)$ for the coordinates, so $q = 0.5 \cdot p + 2$.

In least squares, we generally want to find

$$x^* = \arg\min_{x}|| Ax - b ||_2^2.$$

where $x$ is the vector of unknowns. If we compute the gradient and set it to zero, similar to what we did in the 1D case above, we will get the well-known solution

$$x^* = (A^\top A)^{-1} A^\top b$$

(see e.g. chapter 3 in Hastie et al.). Note that $x$ are our unknowns to be estimated, not the $x$-values along the $x$-axis. That is, $x=(x_0, x_1)^\top = (\mbox{intercept},\mbox{slope})^\top$. Whereas for the points, let us say the point coordinates are $P_i = (p_i, q_i)$.

For a given value $p_i$, along the horizontal axis, the line with intercept $x_0$ and slope $x_1$ has the value $x_1\cdot p_i + x_0$, while the value of the point is $q_i$. So the residual vector is:

$$ r = Ax-b = \begin{bmatrix}1&p_0\\ 1&p_1\\ \vdots & \vdots\\ 1&p_{m-1} \end{bmatrix}\cdot
   \begin{bmatrix}x_0\\x_1\end{bmatrix} - \begin{bmatrix}q_0\\q_1\\ \vdots \\q_{m-1}\end{bmatrix}
$$

and we minimize $||r||_2^2$. Note that $r_i$ is not the perpendicular distance, but rather the ($q$-) axis parallel distance.  Instead of $(A^\top A)^{-1} A^\top$ one typically uses the [pseudoinverse](https://en.wikipedia.org/wiki/Moore%E2%80%93Penrose_inverse) which is typically available as `pinv` (note that although we do it here, one does not have to compute the inverse to solve the equation system).

### First, introduce the data

In [ ]:
# This line is our true model.
def line(x):
    return 0.5*x + 2

# Generate samples: uniform along x, and for each x, additive Gaussian "measurement" noise.
rng = np.random.default_rng(12345) # Restart.
x_2d_vals = np.linspace(0, 10, num=100)
n_2d_meas = 100
inliers_x = rng.uniform(0,10,n_2d_meas)
inliers_y = line(inliers_x) + rng.normal(0,0.1,n_2d_meas)
inliers = np.vstack((inliers_x, inliers_y)).T

plt.plot(x_2d_vals, line(x_2d_vals), c="lightgray")
plt.plot(inliers[:,0], inliers[:,1], "b.")
plt.ylim(0,10);

In [ ]:
# "Symmetric outliers": uniform samples.
n_2d_outliers = 70
outliers_x = rng.uniform(0,10,n_2d_outliers)
outliers_y = line(outliers_x) + rng.uniform(-4,4,n_2d_outliers)
outliers = np.vstack((outliers_x, outliers_y)).T
plt.plot(x_2d_vals, line(x_2d_vals), c="lightgray")
plt.plot(inliers[:,0], inliers[:,1], "b.")
plt.plot(outliers[:,0], outliers[:,1], "r.")
plt.ylim(0,10);

In [ ]:
# "Asymmetric outliers": uniform samples, all above.
outliers_a_y = line(outliers_x) + rng.uniform(0,4,n_2d_outliers)
outliers_a = np.vstack((outliers_x, outliers_a_y)).T
plt.plot(x_2d_vals, line(x_2d_vals), c="lightgray")
plt.plot(inliers[:,0], inliers[:,1], "b.")
plt.plot(outliers_a[:,0], outliers_a[:,1], "r.")
plt.ylim(0,10);

### Now, the estimation results for the least squares case.

First, let us see the solution for the system without outliers.

In [ ]:
def report_l2(inl, outl):
    both = np.concatenate((inl, outl))  # (m x 2) matrix of all point coordinates.

    # Estimate.
    A = np.vstack((np.ones(both.shape[0]), both[:,0])).T
    intercept, slope = np.linalg.pinv(A) @ both[:,1]

    # Report.
    print("intercept %.3f slope %.3f" % (intercept, slope))
    plt.plot(x_2d_vals, line(x_2d_vals), c="lightgray")
    plt.plot(inl[:,0], inl[:,1], "b.")
    plt.plot(outl[:,0], outl[:,1], "r.")
    plt.plot(x_2d_vals, slope*x_2d_vals+intercept, c="k")
    plt.ylim(0,10)

report_l2(inliers,np.empty((0,2)))

Now, with "symmetric" outliers.

In [ ]:
report_l2(inliers, outliers)

And with "asymmetric" outliers:

In [ ]:
report_l2(inliers, outliers_a)

This was as expected. The symmetric noise had a relatively small impact, but with all outliers above the line, the result was substantially shifted.

## Least absolute deviations for the line

Now it gets more interesting. Remember, we now have to do a $l_1$-norm minimization:

$$x^* = \arg\min_x || Ax - b ||_1,$$

which is exactly the same as

$$x^* = \arg\min_x \sum_{i=0}^{m-1} |a_i^\top x-b_i|,$$

were $a_i^\top$ is the $i$-th row vector of $A$.

How can we find this minimum? Being a sum of absolute values, the function is not differentiable. It is also not linear. However, since $|\cdot|$ is a *convex function*, the *sum is convex*, too. So even though there is no closed-form solution, we know we can solve the problem efficiently.

We can re-formulate the goal. Instead of $ \min_x \sum_{i=0}^{m-1} |a_i^\top x-b_i|$ we write

$$ \min_{s,x} \sum_{i=0}^{m-1} s_i,$$

subject to

$$ | a_i^\top x - b_i | \leq s_i .$$

We can get rid of the absolute value by doubling the number of conditions,

$$ \begin{matrix} \phantom{-}(a_i^\top x - b_i) & \leq & s_i & \mbox{and}\\
                 -(a_i^\top x - b_i) & \leq & s_i, & \\ \end{matrix} $$
                 
and so, overall the problem is of the form

$$ \begin{array}{rl} \mbox{minimize the value of:} & c^\top x' \\
               \mbox{among all $x\in\mathbb{R}$ satisfying:} & A' x' \leq b' \end{array}$$
                   
where $x'$, $A'$ and $b'$ are formed from the above entities and "$\leq$" between vectors is understood componentwise.

## Convex optimization and linear programming
The problem we have just formulated is called a **linear program**, and the methods to solve it are subsumed as **linear programming** (although it has nothing to do with "programming" as we understand it today).

They are called *linear*, because both:
- they have a linear **objective function** $f(x) = c^\top x = c_0 x_0+ c_1 x_1 + \ldots + c_{n-1} x_{n-1}$ *and*
- the **constraints** $g(x) = A x \leq b$ **are linear.**

Problems like the one above are special cases of optimization (see Boyd & Vandenberghe):
- **Mathematical optimization** is the general case:

$$ \begin{array}{rl} \mbox{minimize the value of:} & f(x) \\
               \mbox{subject to:} & g(x) \leq b, \end{array}$$
               
- for a **linear program**, the objective and constraints must be linear, i.e. the components (say, $h_i$) of $f$ and $g$ must satisfy:

$$h_i(\alpha x + \beta y) = \alpha h_i(x) + \beta h_i(y),$$

- whereas for a **convex optimization problem**, objective and constraints must satisfy:

$$h_i(\alpha x + \beta y) \leq \alpha h_i(x) + \beta h_i(y).$$

Linear programs and least-squares problems are special cases of convex optimization.

Linear programming problems **can be solved efficiently**, the algorithms used are the famous **simplex method** due to George Dantzig (which however is not polynomial), the **ellipsoid method** (which is polynomial, but much slower in practice), and **interior point methods**, which are widely used nowadays. See also Matousek & Gärtner for a gentle introduction.

To solve a linear programming problem, one uses typically a *solver*. This has the beauty of not having to program a solution to one's problem, instead it suffices to define the equations that describe one's problem.

There are free solvers and commercial ones, most of which are free for academic users. We will use `cvxpy`, which is a modelling language for convex optimization problems. It includes some solvers, but also is able to call external ones, see the list [here](https://www.cvxpy.org/tutorial/advanced/index.html#choosing-a-solver). That is, if you use `cvxpy` and also install a academic version of the commercial solver `gurobi`, then calling `solve()` in `cvxpy` may call an internal solver for "easy" problems and `gurobi` for "harder" problems (as we will see below).

***
Now lets develop the required formulas for our problem above. We have to minimize

$$ \min_{s,x} \sum_{i=0}^{m-1} s_i,$$

where in addition to the two original variables $x=(x_0,x_1)=(\mbox{intercept},\mbox{slope})$, we now also have the $s_i$ variables, one for each condition:

$$ \begin{matrix} \phantom{-}(a_i^\top x - b_i) & \leq & s_i & \mbox{and}\\
                 -(a_i^\top x - b_i) & \leq & s_i, & \\ \end{matrix} $$
                 
if we sort that into variables (l.h.s.) and constants (r.h.s.) we get

$$ \begin{matrix} \phantom{-}a_i^\top x - s_i & \leq & \phantom{-}b_i & \mbox{and}\\
                 -a_i^\top x - s_i & \leq & -b_i, & \\ \end{matrix} $$

and if we define, as above, $A$ and $b$ as

$$ A = \begin{bmatrix}1&p_0\\ 1&p_1\\ \vdots & \vdots\\ 1&p_{m-1} \end{bmatrix},
   b = \begin{bmatrix}q_0\\q_1\\ \vdots \\q_{m-1}\end{bmatrix} $$
   
then our linear program is:

$$ \begin{array}{rl} \mbox{minimize:} & \sum_i s_i \\
               \mbox{subject to:} & \phantom{-}A x - s \leq b \\
                                  & -A x - s \leq -b\end{array}$$

and if you check the Python program below, this is pretty much how we can express it in `cvxpy`. Instead of the sum, one can alternatively write

$$ \begin{array}{rl} \mbox{minimize:} & ||s||_1 \\
               \mbox{subject to:} & \phantom{-}A x - s \leq b \\
                                  & -A x - s \leq -b\end{array}$$

therefore, we *minimize the* $l_1$ *norm of* $s$.

In [ ]:
import cvxpy as cp

def report_l1(inl, outl):
    both = np.concatenate((inl, outl))  # (m x 2) matrix of all point coordinates.

    # Estimate.
    m = both.shape[0]
    A = np.vstack((np.ones(m), both[:,0])).T
    b = both[:,1]
    # Setup problem for solver and solve.
    x, s = cp.Variable(2), cp.Variable(m)
    prob = cp.Problem(cp.Minimize(sum(s)),
                      [ A @ x - s <=  b,
                       -A @ x - s <= -b])
    prob.solve(verbose=False)  # Set to true to see solver working.
    
    # Print and plot results.
    intercept, slope = x.value
    print("intercept %.3f slope %.3f" % (intercept, slope))
    plt.plot(x_2d_vals, line(x_2d_vals), c="lightgray")
    plt.plot(inl[:,0], inl[:,1], "b.")
    plt.plot(outl[:,0], outl[:,1], "r.")
    plt.plot(x_2d_vals, slope*x_2d_vals+intercept, c="k")
    plt.ylim(0,10)

report_l1(inliers,np.empty((0,2)))

In [ ]:
report_l1(inliers, outliers)

In [ ]:
report_l1(inliers, outliers_a)

However if we raise the number of outliers, this will break it, just as in the 1D case.

In [ ]:
n_2d_more_outliers = 200
more_outliers_x = rng.uniform(0,10,n_2d_more_outliers)
more_outliers_a_y = line(more_outliers_x) + rng.uniform(0,4,n_2d_more_outliers)
more_outliers_a = np.vstack((more_outliers_x, more_outliers_a_y)).T
report_l1(inliers, more_outliers_a)

## Maximum Consensus for the line

Remember for maximum consensus, we have to count $+1$ if an observation is within bounds $\pm\varepsilon$. We then have to maximize the count of inliers

$$x^* = \arg\max_x \sum_i \mathbb{1}\left( |a_i^\top x - b_i| \leq \varepsilon \right).$$

In the 1D case, we have expressed this already in an alternative way, namely as a minimization of the count of outliers. Define the $l_0$ (pseudo-) norm being *the number of non-zero elements*, i.e., $||s||_0$ is the count of elements of the vector $s$ which are non-zero. We now use $s$ as a *slack* variable, for which entry $s_i$ indicates the amount by which observation $i$ violates the $\leq\varepsilon$ constraint. Then, maximum consensus is the problem to

$$ \begin{array}{rl} \mbox{minimize:} & ||s||_0 \\
               \mbox{subject to:} & | a_i^\top x - b_i | \leq \varepsilon + s_i \\
                                  & s_i \geq 0.\end{array}$$

Thus, if observation $i$ is an inlier, it satisfies $| a_i^\top x - b_i | \leq \varepsilon$, and the slack $s_i$ will be zero. However, observation $i$ is an outlier, $s_i > 0$ must hold, and this increases $||s||_0$ by $1$. Since the behaviour of $||s||_0$ is hard to handle, one typically uses the **big-M** method to re-formulate this as follows:

$$ \begin{array}{rl} \mbox{minimize over $x\in\mathbb{R}^n$ and $z\in\{0,1\}^m$:} & \sum_i z_i \\
               \mbox{subject to:} & | a_i^\top x - b_i | \leq \varepsilon + z_i\cdot M, \end{array}$$

where $M$ is a sufficiently large *(big)* constant, and the variables $z_i$ are binary. They switch on or off the "outlier status" of the $i$-th observation. If $z_i=0$, the constraint $| a_i^\top x - b_i | \leq \varepsilon$ must hold, and observation $i$ is an inlier. However, if $z_i=1$, $| a_i^\top x - b_i | \leq \varepsilon + M$ must hold, and since $M$ is very large, this is always satisfied.

Since the $z_i$ are required to be binary (or, they could also be from $\mathbb{Z}$, we left the world of *linear programming* and entered the world of **Integer Programming (IP)**. More accurately, since we have to optimize a mixture of variable types, $x\in\mathbb{R}^n$ *and* $z\in\{0,1\}^m$, this is a **Mixed Integer Program (MIP)**.

Going from $\mathbb{R}$ to $\{0,1\}$ or $\mathbb{Z}$ seems like a little variation, but it changes everything. In general, there are no efficient ways to solve IP and MIP problems -- they are NP-hard. I.e., in general, one has to search the solution space. Well-known methods are *branch-and-bound*, *cutting planes*, and *branch-and-cut*. These methods are employed by solvers.

Despite being hopeless in general, some IP/MIP are efficiently solvable, for others there are approximation algorithms (with guarantees) or efficient heuristics (without guarantees), see e.g. Conforti et al.. There are also links to combinatorial optimization, see Korte & Vygen. Approximation algorithms are discussed in Williamson & Shmoys.

Side note on big-M: Although mathematically elegant, the "big-M" method is troubled, because if $M$ is made too small, the results will be wrong, and if $M$ is too large, convergence is slow and numerical instabilities may occur. This can be solved using *Mixed Logical Programming*, so in some solvers, one can use so-called *indicator constraints* of the form: $s_i \Rightarrow cond$, "if $s_i$ is true, condition $cond$ must hold" as a replacement for the big-M formulation.

In the following, we write down the equations from above in `cvxpy` notation:

In [ ]:
def report_l0(inl, outl, eps, bigM):
    both = np.concatenate((inl, outl))  # (m x 2) matrix of all point coordinates.

    # Estimate
    m = both.shape[0]
    A = np.vstack((np.ones(m), both[:,0])).T
    b = both[:,1]
    x, z = cp.Variable(2), cp.Variable(m, boolean=True)  # Note: boolean=True.
    prob = cp.Problem(cp.Minimize(sum(z)),
                      [ A @ x - z * bigM <= eps + b,
                       -A @ x - z * bigM <= eps - b])
    prob.solve(verbose=True)  # Set to true to watch the solver working.
    
    # Print and plot results.
    intercept, slope = x.value
    print("intercept %.3f slope %.3f" % (intercept, slope))
    plt.plot(x_2d_vals, line(x_2d_vals), c="lightgray")
    plt.plot(inl[:,0], inl[:,1], "b.")
    plt.plot(outl[:,0], outl[:,1], "r.")
    plt.plot(x_2d_vals, slope*x_2d_vals+intercept, c="k")
    plt.ylim(0,10)

# Uncomment the next line to run (may take two minutes).
#report_l0(inliers, more_outliers_a, eps=0.2, bigM=4.5)

***
Leaving the robust optimization topic!
***

# Convex optimization example: *model selection*, aka *shrinkage*

Now we used the problem of robust estimation to motivate least squares, least absolute deviation, and maximum consensus. To find a solution, we used an analytical solution (least squares), linear programming (least absolute deviation), and mixed integer programming (maximum consensus).

Although we motivated all this using the case of robust estimation, these techniques are important in their own right and are applied in numerous areas. This section is an example where you would encounter quadratic programming, the second-order sibling of linear programming.

The *model selection problem* occurs almost everywhere: if we want to explain data using a model, we should make the model sufficiently complex so that it is able to describe the "main characteristics" of the data. However, we should not make it overly complex, because then it will learn by heart the particular samples we have.

Here, we will illustrate this with fitting a polynomial to data. Intuitively, the "model complexity" is tied to the degree of the polynomial, or the number of monomials used to describe the polynomial (see below).

Methods for *model selection* are also termed *shrinkage methods*. See e.g. chap. 3.4 in Hastie et al.

We will look at four different methods to select our model:
- variation of the degree of the polynomial,
- ridge regression,
- the lasso,
- monomial selection via IP.

*Note: in the following, the intercept is not estimated. If we would estimate this as well, then we would have an additional "$1$" column in A, and $x_0$ would be the corresponding coefficient to be estimated. However then, in the ridge regression and lasso approaches below, one would have to replace $||x||_2$ and $||x||_1$ by $||x_{1:k-1}||_2$ and $||x_{1:k-1}||_1$, respectively, because $x_0$ depends on the choice of the coordinate system and it would make no sense to suppress large values. Equivalently, one can estimate the mean of $x$ and $b$ and work with centered inputs from there on, see Hastie et al., chap. 4.3.1 and 3.4.2.*

In [ ]:
# This function is our true model.
def model(x):
    return x**3 - 3*x

# Generate samples: uniform along x, and for each x, additive Gaussian "measurement" noise.
rng = np.random.default_rng(1234) # Restart.
x_2d_vals = np.linspace(-2,2,num=100)
n_2d_train, n_2d_test = 10, 10
data_x = rng.uniform(-2,2,n_2d_train+n_2d_test)
data_y = model(data_x) + rng.normal(0,0.2,data_x.size)
data = np.vstack((data_x, data_y)).T
train = data[:n_2d_train,:]
test  = data[n_2d_train:,:]

plt.plot(x_2d_vals, model(x_2d_vals), c="lightgray")
plt.plot(train[:,0], train[:,1], "b.", label="train")
plt.plot(test[:,0], test[:,1], "r.", label="test")
plt.legend()
ylim=(-4,4)
plt.ylim(ylim);

## Finding a suitable model via degree variation

As above, we use least-squares to find the parameters of the polynomial

$$x^* = \arg\min_{x}|| Ax - b ||_2^2.$$

which leads to the well-known solution

$$x^* = (A^\top A)^{-1} A^\top b.$$

Varying the "model complexity" is done by changing the degree of the polynomial which means that our $x$ vector has more or less entries. For example, if the degree of the polynomial is $1$, $x$ is a scalar, and $A$ has only one column.

In [ ]:
# Powers of polynomials used for the fit: x**1, x**2,...
powers = range(1,10)

# The A matrix, each column is a x^i of the x coordinates.
A = np.vstack(tuple([train[:,0]**p for p in powers])).T
A_test = np.vstack(tuple([test[:,0]**p for p in powers])).T

# This is the least squares estimate.
coeff = np.linalg.pinv(A) @ train[:,1]
print("coefficients =", coeff)
    
# Plot result. Helper matrix M is like A, but for "all" x values.
M = np.vstack(tuple([x_2d_vals**p for p in powers])).T
plt.plot(x_2d_vals, model(x_2d_vals), c="lightgray")  # The model.
plt.plot(x_2d_vals, M@coeff, c="k")  # The estimate.
plt.plot(train[:,0], train[:,1], "b.")  # The training data used for the estimate.
plt.plot(test[:,0], test[:,1], "r.")  # The test data, just for visualization.
plt.ylim(ylim);

In [ ]:
# LS where degree can be set via slider.
import ipywidgets as ipyw
def plot_scan(max_columns):
    coeff = np.linalg.pinv(A[:,:max_columns]) @ train[:,1]
    plt.plot(x_2d_vals, model(x_2d_vals), c="lightgray")  # The model.
    plt.plot(x_2d_vals, M[:,:max_columns]@coeff, c="k")  # The estimate.
    plt.plot(train[:,0], train[:,1], "b.")  # The training data used for the estimate.
    plt.plot(test[:,0], test[:,1], "r.")  # The test data, just for visualization.
    plt.title(" | ".join("%.2f" % c for c in coeff))
    plt.ylim(ylim)
ipyw.interact(plot_scan, max_columns = ipyw.IntSlider(min=1, max=A.shape[1], value=A.shape[1]));

In [ ]:
# Plot training/test vs. degree.
errors = []
for max_columns in range(len(powers),0,-1):
    coeff = np.linalg.pinv(A[:,:max_columns]) @ train[:,1]  # LS for this degree.
    train_error = np.mean((A[:,:max_columns]@coeff - train[:,1])**2)  # MSE
    test_error = np.mean((A_test[:,:max_columns]@coeff - test[:,1])**2)
    errors.append((train_error, test_error))
degree_errors = np.array(errors)
degree_min_i = np.argmin(degree_errors[:,1])
print("minimum test error at degree = %d" % powers[A.shape[1]-1-degree_min_i])
print("coefficients = ", np.linalg.pinv(A[:,:A.shape[1]-degree_min_i]) @ train[:,1])
print("test error (MSE) = %.4f" % degree_errors[degree_min_i,1])

plt.axvline(x=degree_min_i, c="gray", ls="--")
plt.plot(degree_errors[:,0], c="blue", label="train")
plt.plot(degree_errors[:,1], c="red", label="test")
plt.xticks(range(len(powers)), range(len(powers),0,-1));
plt.legend()
train_test_ylim = (0,8)
plt.ylim(train_test_ylim);

## Finding a suitable model via ridge regression

Instead finding the solution for all different degrees (and dimensions of $A$), we "suppress" model complexity by forcing $x$ to be small (in terms of the squared $l_2$ norm). This is called *regularization*.

$$x^* = \arg\min_{x}|| Ax - b ||_2^2 + \lambda ||x||_2^2.$$

where $\lambda$ is a hyperparameter. Fortunately, this modification integrates nicely with standard least-squares, and the closed-form solution is

$$x^* = (A^\top A + \lambda I)^{-1} A^\top b,$$

so all there is to do is to add $\lambda$ to the main diagonal. This is a standard trick to remove singularities. Also, in surveying this is typically used as a soft constraint to "fix parameters by pseudo observation". And, in ML estimation, it can be nicely motivated by a prior distribution for x.

Varying the "model complexity" now means to play with $\lambda$. Intuitively, if it is 0, we obtain the standard least-squares solution, with the most complex model. If it grows, the model will become simpler.

In [ ]:
# Ridge regression.
def ls_ridge(A, b, la):
    return np.linalg.solve(A.T @ A + np.eye(A.shape[1])*la, A.T @ b)

coeff = ls_ridge(A, train[:,1], 0.0001) # Set lambda here.
print(coeff)

# Plot result.
plt.plot(x_2d_vals, model(x_2d_vals), c="lightgray")
plt.plot(x_2d_vals, M@coeff, c="k")
plt.plot(train[:,0], train[:,1], "b.")
plt.plot(test[:,0], test[:,1], "r.")
plt.ylim(ylim);

In [ ]:
# Interactive: estimation vs. lambda.
lambda_logrange = (-11, 2)
def plot_scan(Lambda):
    coeff = ls_ridge(A, train[:,1], Lambda)
    plt.plot(x_2d_vals, model(x_2d_vals), c="lightgray")
    plt.plot(x_2d_vals, M@coeff, c="k")
    plt.plot(train[:,0], train[:,1], "b.")
    plt.plot(test[:,0], test[:,1], "r.")
    plt.title(" | ".join(("%.2f" % c) if abs(c) > 1e-2 else "****" for c in coeff))
    plt.ylim(ylim)
ipyw.interact(plot_scan, Lambda = ipyw.FloatLogSlider(
    min=lambda_logrange[0], max=lambda_logrange[1], value=lambda_logrange[0]));

In [ ]:
# Train and test error vs. lambda.
lambdas = np.logspace(*lambda_logrange,100)
errors, components = [], []
for la in lambdas:
    coeff = ls_ridge(A, train[:,1], la)  # Ridge regression for this lambda.
    train_error = np.mean((A@coeff - train[:,1])**2)  # MSE
    test_error = np.mean((A_test@coeff - test[:,1])**2)
    errors.append((train_error, test_error))
    components.append((abs(coeff)>1e-2).sum())  # Also store the number of non-zero components.
ridge_errors = np.array(errors)
ridge_min_i = np.argmin(ridge_errors[:,1])
print("minimum test error at lambda = %.3f" % lambdas[ridge_min_i])
print("coefficients = ", ls_ridge(A, train[:,1], lambdas[ridge_min_i]))
print("test error (MSE) = %.4f" % ridge_errors[ridge_min_i,1])

plt.axvline(x=lambdas[ridge_min_i], c="gray", ls="--")
plt.plot(lambdas, components, ":", c="lightgray", label="components")
plt.plot(lambdas, ridge_errors[:,0], c="blue", label="train")
plt.plot(lambdas, ridge_errors[:,1], c="red", label="test")
plt.xscale("log")
plt.legend()
train_test_ylim = (0,10)
plt.ylim(train_test_ylim)

## Finding a suitable model via the lasso method

Now, we use the $l_2$ norm for the residuals, but the $l_1$ norm to "suppress" model complexity:

$$x^* = \arg\min_{x}|| Ax - b ||_2^2 + \lambda ||x||_1.$$

where as before, $\lambda$ is a hyperparameter. This does not integrate nicely in the least squares approach. What we obtain is a quadratic programming problem, which we will hand over to the solver. $\lambda$ plays the same role as before.

In [ ]:
# Lasso. Not linear programming, but rather quadratic programming (and convex optimization).
def loss_fn(A, b, x):
    return cp.norm2(A @ x - b)**2
def regularizer(x):
    return cp.norm1(x)
def objective_fn(A, b, x, Lambda):
    return loss_fn(A, b, x) + Lambda * regularizer(x)
def mse(A, b, x):
    return (1.0 / A.shape[0]) * loss_fn(A, b, x).value

def ls_lasso(A, b, Lambda):
    x = cp.Variable(A.shape[1])
    #lambd = cp.Parameter(nonneg=True)
    problem = cp.Problem(cp.Minimize(objective_fn(A, b, x, Lambda)))
    problem.solve(solver="GUROBI", verbose=False)
    return x.value
ls_lasso(A, train[:,1], 1.0)

In [ ]:
# Interactive: estimation vs. lambda.
def plot_scan(Lambda):
    coeff = ls_lasso(A, train[:,1], Lambda)
    plt.plot(x_2d_vals, model(x_2d_vals), c="lightgray")
    plt.plot(x_2d_vals, M@coeff, c="k")
    c2 = coeff.copy()
    c2[abs(c2)<1e-2]=0
    plt.plot(x_2d_vals, M@c2, "--", c="k")
    plt.plot(train[:,0], train[:,1], "b.")
    plt.plot(test[:,0], test[:,1], "r.")
    plt.title(" | ".join(("%.2f" % c) if abs(c) > 1e-2 else "****" for c in coeff))
    plt.ylim(ylim)
ipyw.interact(plot_scan, Lambda = ipyw.FloatLogSlider(
    min=lambda_logrange[0], max=lambda_logrange[1], value=lambda_logrange[0]));

In [ ]:
# Train and test error vs. lambda.
errors, components = [], []
for la in lambdas:
    coeff = ls_lasso(A, train[:,1], la)  # Ridge regression for this lambda.
    train_error = np.mean((A@coeff - train[:,1])**2)  # MSE
    test_error = np.mean((A_test@coeff - test[:,1])**2)
    errors.append((train_error, test_error))
    components.append((abs(coeff)>1e-2).sum())  # Also store the number of non-zero components.
    
lasso_errors = np.array(errors)
lasso_min_i = np.argmin(lasso_errors[:,1])
print("minimum test error at lambda = %.3f" % lambdas[lasso_min_i])
print("coefficients = ", ls_lasso(A, train[:,1], lambdas[lasso_min_i]))
print("test error (MSE) = %.4f" % lasso_errors[lasso_min_i,1])

plt.axvline(x=lambdas[lasso_min_i], c="gray", ls="--")
plt.plot(lambdas, components, ":", c="lightgray", label="components")
plt.plot(lambdas, lasso_errors[:,0], c="blue", label="train")
plt.plot(lambdas, lasso_errors[:,1], c="red", label="test")
plt.xscale("log")
plt.legend()
plt.ylim(train_test_ylim);

In [ ]:
# Comparison of test errors of ridge regression vs. lasso.
plt.plot(lambdas, ridge_errors[:,0], c="lightblue", label="train (ridge)")
plt.plot(lambdas, ridge_errors[:,1], c="lightsalmon", label="test (ridge)")
plt.plot(lambdas, lasso_errors[:,0], c="blue", label="train (lasso)")
plt.plot(lambdas, lasso_errors[:,1], c="red", label="test (lasso)")
plt.xscale("log")
plt.legend()
plt.ylim(train_test_ylim);

### Comparing the coefficients: ridge regression vs. lasso.
For ridge regression and lasso, we have a continuous parameter $\lambda$ to influence the model complexity. One can draw the components of $x$ vs. $\lambda$, sometimes called *regularization path*.

Comparing the results of ridge regression vs. lasso, one can see the tendency of lasso to obtain zero coefficients, or *sparse* results.

In [ ]:
# Ridge: coefficients vs. lambda.
ridge_results = np.array([ ls_ridge(A, train[:,1], la) for la in lambdas ])
for c in range(ridge_results.shape[1]):
    plt.plot(lambdas, ridge_results[:,c], label=powers[c])
plt.legend()
plt.xscale("log")

In [ ]:
# Ridge: coefficients vs. lambda.
plt.axhline(y=-2, c="gray", ls="--")
ridge_results = np.array([ ls_ridge(A, train[:,1], la) for la in lambdas ])
for c in range(ridge_results.shape[1]):
    plt.plot(lambdas, np.log10(np.abs(ridge_results[:,c])+1e-7), label=powers[c])
plt.legend()
ylim_log_coeff=(-7.5,3)
plt.ylim(ylim_log_coeff)
plt.xscale("log")

In [ ]:
# Lasso: coefficients vs. lambda.
lasso_results = np.array([ ls_lasso(A, train[:,1], la) for la in lambdas ])
for c in range(lasso_results.shape[1]):
    plt.plot(lambdas, lasso_results[:,c], label=powers[c])
plt.legend()
plt.xscale("log")

In [ ]:
# Lasso: coefficients vs. lambda.
plt.axhline(y=-2, c="gray", ls="--")
lasso_results = np.array([ ls_lasso(A, train[:,1], la) for la in lambdas ])
for c in range(lasso_results.shape[1]):
    plt.plot(lambdas, np.log10(np.abs(lasso_results[:,c])+1e-7), label=powers[c])
plt.legend()
plt.xscale("log")
plt.ylim(ylim_log_coeff);

In [ ]:
# Both coefficient sets in one figure.
for c in range(ridge_results.shape[1]):
    plt.plot(lambdas, np.log10(np.abs(ridge_results[:,c])+1e-7), c="silver")
for c in range(lasso_results.shape[1]):
    plt.plot(lambdas, np.log10(np.abs(lasso_results[:,c])+1e-7), c="k")
plt.xscale("log")
plt.ylim(ylim_log_coeff);

## Monomial selection via IP

Let's come back to the first technique we looked at, *degree variation*. We selected the number of components $1\leq c\leq k$ and then computed the least squares estimate for a polynomial of degree $c$.

For example, in our case, the reference model is of degree $3$, and we obtained the best result using a polynomial of degree $3$. However, the reference model only contains the monomials $x$ and $x^3$, whereas our fit contains *all monomials up to degree* $3$, i.e., $x$, $x^2$, $x^3$.

So instead of requiring a maximum degree for the polynomial, we could require a maximum number of components (monomials). For example, if our reference model would contain only the odd monomials $x$, $x^3$, $x^5$, $x^7$, we could obtain a good fit using a polynomial with 4 components, but we would not achieve a good fit with a polynomial of degree 4, since this would contain $x$, $x^2$, $x^3$, $x^4$. Thus, we would include "useless" monomials $x^2$, $x^4$, while being unable to model the effects of $x^5$ and $x^7$.

The best fit with at most $c$ components can be elegantly formulated using integers (a mixed integer quadratic program):

$$ \begin{array}{rl} \mbox{minimize:} & ||Ax - b||_2^2 \\
               \mbox{subject to:} & | x_i | \leq z_i\cdot M \\
                                  & \sum_i z_i \leq c,\end{array}$$

$x_i\in\mathbb{R}$, $z_i\in \{0,1\}$, again using the "big-M" formulation. If $z_i$ is $0$, the component $x_i$ is forced to $0$ in the least squares estimation, otherwise, it is unconstrained. This can be directly formulated in `cvxpy`:

In [ ]:
# Compute solutions with 1,...,c components.
# If you encounter problems with this bigM constant, try to play with it. For me, this value
# worked on Linux, but not on Windows, a kind of worrying result. But after all, bigM
# is known for being numerically problematic. The solution would be to re-formulate it in terms
# of "indicator constraints".
bigM = 5000
k_comp_solutions = [ None ]
for c in range(1, A.shape[1]+1):
    x, z = cp.Variable(A.shape[1]), cp.Variable(A.shape[1], boolean=True)
    prob = cp.Problem(cp.Minimize(cp.norm2(A @ x - train[:,1])**2),
                      [-bigM*z <= x,
                       x <= bigM*z,
                       cp.sum(z) <= c ])
    prob.solve(verbose=False)
    k_comp_solutions.append(x.value)

# k component solutions via slider.
import ipywidgets as ipyw
def plot_scan(max_components):
    coeff = k_comp_solutions[max_components]
    plt.plot(x_2d_vals, model(x_2d_vals), c="lightgray")  # The model.
    plt.plot(x_2d_vals, M@coeff, c="k")  # The estimate.
    plt.plot(train[:,0], train[:,1], "b.")  # The training data used for the estimate.
    plt.plot(test[:,0], test[:,1], "r.")  # The test data, just for visualization.
    plt.title(" | ".join(("%.2f" % c) if abs(c) > 1e-2 else "****" for c in coeff))
    plt.ylim(ylim)
ipyw.interact(plot_scan, max_components = ipyw.IntSlider(min=1, max=A.shape[1], value=A.shape[1]));

In [ ]:
# Plot training/test vs. number of components.
errors = []
for max_columns in range(1, A.shape[1]+1):
    coeff = k_comp_solutions[max_columns]
    train_error = np.mean((A@coeff - train[:,1])**2)  # MSE
    test_error = np.mean((A_test@coeff - test[:,1])**2)
    errors.append((train_error, test_error))
component_errors = np.array(errors)
best_components = np.argmin(component_errors[:,1])+1
print("minimum test error at #components = %d" % best_components)
print("coefficients = ", k_comp_solutions[best_components])
print("test error (MSE) = %.4f" % component_errors[best_components-1,1])

plt.axvline(x=A.shape[1]-best_components, c="gray", ls="--")
plt.plot(range(A.shape[1]-1,-1,-1), component_errors[:,0], c="blue", label="train")
plt.plot(range(A.shape[1]-1,-1,-1), component_errors[:,1], c="red", label="test")
plt.xticks(range(A.shape[1]), range(A.shape[1],0,-1))
plt.legend()
plt.ylim(train_test_ylim);

# A "typical" integer programming problem

Often, this is combinatorial optimization which can be expressed as an IP problem. Problems like:
- assignment problem (connect taxi drivers to customers)
- transportation/ transshipment problem
- knapsack problem
- bin packing problem
- cutting stock problem
- ...

But also in Geoinformatics:
- generalization
- label placement
- maximum a posteriori estimation in case of discrete random variables (see Chap. 13.5, *MAP as a linear optimization problem* in Koller & Friedman).

Here we will pick the following one:

## The uncapacitated facility location problem

We have clients $D$ (demand) and tentative outlet locations $F$ (facilities). If client $j$ goes to outlet $i$, this is associated with a cost $c_{ij}$, here we will use the Euclidean distance. So the client will always go to the closest outlet. If we want happy clients, we would open up as many outlets as possible (i.e. at each tentative location). However, to open an outlet up, we have to build it, so there is a one-time fixed cost $f_i$ to open outlet $i$. Now the goal is to minimize the total cost.

Formulated as an IP problem, we have the variables:
- $y_i$ which are binary variables telling if outlet $i$ is opened up ($1$) or not ($0$).
- $x_{ij}$ which are binary variables telling if customer $j$ goes to outlet $i$.

Then, the optimization problem is:

$$ \begin{array}{rlll} \mbox{minimize:} & \sum_i f_i y_i + \sum_{ij} c_{ij} x_{ij}\\
               \mbox{subject to:} & \sum_i x_{ij} = 1 & \forall j &
                                      \mbox{(each client goes to exactly one outlet)}\\
                                  & x_{ij} \leq y_i & \forall i,j &
                                      \mbox{(a client can only go to an outlet that is open)}\\
                                  & x_{ij}, y_i \in \{0,1\}.
                                  \end{array}$$

This is a typical IP formulation: the binary variables are used as switches.

The *uncapacitated facility location problem* is a very well researched problem (see Korte & Vygen, chap. 22.1, and Williamson & Shmoys).

In [ ]:
from scipy.spatial import cKDTree
from matplotlib.collections import LineCollection
rng = np.random.default_rng(12345) # Restart.
n_factories = 40
n_demands = 80
factory_locations, demand_locations = rng.uniform(0,1,(n_factories, 2)), rng.uniform(0,1,(n_demands, 2))

# Random factory costs.
factory_max_cost = 20
factory_costs_1 = rng.uniform(0,factory_max_cost,n_factories) 
factory_costs_2 = factory_max_cost - factory_costs_1 

def evaluate(f_cost = 0.5, radius = 0.1):
    # Setup possible pairs, according to max_radius.
    tree = cKDTree(factory_locations)
    close_factories = tree.query_ball_point(demand_locations, radius)
    edges = np.array([ (i,n) for i, cf in enumerate(close_factories) for n in cf ], dtype=int)
    incoming_indices, j = [], 0
    for nlist in close_factories:
        l = len(nlist)
        if l > 0:
            incoming_indices.append((j,j+l))
            j += l

    # Compute edge costs, use Euclidean distance.
    edge_costs = np.array([ np.linalg.norm(demand_locations[d]-factory_locations[f]) for d,f in edges ])

    # Factory costs are a linear combination of the two costs.
    factory_costs = (1.0-f_cost)*factory_costs_1 + f_cost*factory_costs_2

    # Solve problem.
    yi = cp.Variable(n_factories, boolean=True)
    xij = cp.Variable(len(edges), boolean=True)
    prob = cp.Problem(cp.Minimize(factory_costs.T @ yi + edge_costs @ xij),
                      [ cp.sum(xij[f:t]) == 1 for f,t in incoming_indices ] +
                      [ xij <= yi[edges[:,1]] ])

    prob.solve(verbose=False)

    # Plot all possible edges.
    line_coll_data = np.array([ [tuple(demand_locations[d]), tuple(factory_locations[f])] for d, f in edges ])
    fig, ax = plt.subplots()
    ax.add_collection(LineCollection(line_coll_data, colors="gainsboro", zorder=0))
    ax.add_collection(LineCollection(line_coll_data[xij.value > 0], colors="blue", zorder=1))
    # Plot factory and demand locations.
    plt.scatter(demand_locations[:,0], demand_locations[:,1], c="white", edgecolors="k", s=50)
    plt.scatter(factory_locations[:,0], factory_locations[:,1], c=factory_costs,
                cmap="magma", vmin=0, vmax=factory_max_cost, s=100, marker="v")
    plt.scatter(factory_locations[:,0][yi.value>0], factory_locations[:,1][yi.value>0], c=factory_costs[yi.value>0],
                cmap="magma", vmin=0, vmax=factory_max_cost, s=300, marker="s", edgecolors="k")

ipyw.interact(evaluate, f_cost = ipyw.FloatSlider(min=0, max=1, step=0.05, value=0.5, continuous_update=False),
              radius = ipyw.FloatSlider(min=0.05, max=1.0, step=0.01, value=0.05, continuous_update=False));

# Bibliography

- Boyd & Vandenberghe: Convex Optimization, [free web download from the authors](https://web.stanford.edu/~boyd/cvxbook/).
- Conforti, Cornuejols, Zambelli: Integer Programming, [Springer](https://www.springer.com/gp/book/9783319110073).
- Hastie, Tibshirani, Friedman: The Elements of Statistical Learning, [free web download from the authors](https://web.stanford.edu/~hastie/ElemStatLearn/).
- Koller & Friedman: Probabilistic Graphical Models, [MIT Press](https://mitpress.ublish.com/book/probabilistic-graphical-models).
- Korte & Vygen: Combinatorial Optimization (also available as German book), [available online for LUH members](https://www.springer.com/gp/book/9783662560389).
- Matousek & Gärtner: Understanding and Using Linear Programming, [available online for LUH members](https://www.springer.com/de/book/9783540306979#otherversion=9783540307174).
- Williamson & Shmoys: The Design of Approximation Algorithms, [free web download from the authors](http://www.designofapproxalgs.com/).

# Preliminaries to run this notebook

- If you don't have Python you will typically install the [anaconda](https://www.anaconda.com/products/individual) distribution. This includes python3, numpy, scipy, matplotlib, etc. On Linux, `pip install` works too.
- To run the notebook itself, start *jupyter notebook* or *JupyterLab* (also included in the anaconda distribution).
- If the interactive sliders do not work, you will have to install ipywidgets, see [here](https://ipywidgets.readthedocs.io/en/stable/user_install.html). It's just a `conda install` or a `pip install`.
- If cvxpy is missing, see [here](https://www.cvxpy.org/install/index.html). This is also just a `conda install` (or a `pip install`).
- The gurobi solver used for some of the cells is a commercial product, but there is a free academic license. You may install it using `conda install gurobi`. Once you have installed it, cvxpy will automatically select it for some of the problems, or if you request it by `problem.solve(solver="GUROBI")`. After installation, you will have to get an *individual academic license* to make it work, follow the instructions [here](https://www.gurobi.com/academia/academic-program-and-licenses/).